In [1]:
# Load Libraries
import warnings
import keras
import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.layers import Input, ELU, Embedding, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Embedding, LSTM
from keras.layers.core import Dense, Dropout, Flatten

from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

Using TensorFlow backend.


In [2]:
# LSTM Model
def lstm(max_len=80, emb_dim=32, max_vocab_len=128, W_reg=regularizers.l2(1e-4)):
    """LSTM with Attention model with the Keras Sequential model"""

    input = Input(shape=(max_len,), dtype='int32', name='lstm_input')
    
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.2)(emb)
    
    lstm = LSTM(units=128, return_sequences=True)(emb)
    lstm = Dropout(0.5)(lstm)
    lstm = Flatten()(lstm)
    
    h1 = Dense(8576, activation='relu')(lstm)
    h1 = Dropout(0.5)(h1)
    
    output = Dense(1, activation='sigmoid', name='lstm_output')(h1)
    
    model = Model(input=[input], output=[output])
   
    return model

In [3]:
x_train, x_test, y_train, y_test = Preprocessor.load_data_binary(10000)

                                                       url    label  class
895657   http://umquartodecena.com/EN_en/xerox/Inv/ziol...  malware      1
765141   https://informapp.in/xvyf69e/v8vq6-h7ekfy-wliy...  malware      1
1098599                       http://92.115.94.82:16516/.i  malware      1
252608   http://185.244.25.194:80/dwabniduawdbwad/dedho...  malware      1
474754   http://kidplearn.co.th/US/scan/qMrqi-Er_VlSOjH...  malware      1
...                                                    ...      ...    ...
103245             http://138.197.215.81/salviatelnet.arm4  malware      1
76496    https://icshongkong.com/wp-content/uploads/202...  malware      1
322150                  http://111.42.102.122:49624/Mozi.m  malware      1
462579              http://207.154.193.227/bins/Alaric.ppc  malware      1
652731              http://dq9wq1wdq9wd1.com/HUN/uho8.yarn  malware      1

[5000 rows x 3 columns]
x_train:  (8000, 80)
y_train:  (8000, 1)
x_test:  (2000, 80)
y_test:  (2000

In [4]:
epochs = 5
batch_size = 64

model = lstm()
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.BinaryAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

Instructions for updating:
Colocations handled automatically by placer.


c:\users\hyojong\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=128, output_dim=32, input_length=80, embeddings_regularizer=<keras.reg...)`
  import sys


Instructions for updating:
Use tf.cast instead.


c:\users\hyojong\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


Train on 7120 samples, validate on 880 samples
Epoch 1/5
7120/7120 [==============================] - 110s 15ms/step - loss: 0.4279 - accuracy: 0.7965 - binary_accuracy: 0.6976 - precision: 0.8223 - recall: 0.6166 - fmeasure: 0.6717 - val_loss: 0.2012 - val_accuracy: 0.9136 - val_binary_accuracy: 0.8036 - val_precision: 1.0000 - val_recall: 0.9141 - val_fmeasure: 0.9548
Epoch 2/5
7120/7120 [==============================] - 108s 15ms/step - loss: 0.2248 - accuracy: 0.9060 - binary_accuracy: 0.8361 - precision: 0.9034 - recall: 0.8839 - fmeasure: 0.8904 - val_loss: 0.2616 - val_accuracy: 0.8886 - val_binary_accuracy: 0.8561 - val_precision: 1.0000 - val_recall: 0.8880 - val_fmeasure: 0.9403
Epoch 3/5
7120/7120 [==============================] - 108s 15ms/step - loss: 0.1761 - accuracy: 0.9288 - binary_accuracy: 0.8686 - precision: 0.9299 - recall: 0.9082 - fmeasure: 0.9167 - val_loss: 0.1288 - val_accuracy: 0.9545 - val_binary_accuracy: 0.8805 - val_precision: 1.0000 - val_recall: 0.955

In [5]:
model_json = model.to_json()
with open("./saved_model/lstm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/lstm.h5")